# Generacion de cubo de tierra (datos)

In [2]:
!pip install noise

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import noise

def generate_terrain():
    shape = (20, 20, 20)
    scale = 100.0
    octaves = 6
    persistence = 0.5
    lacunarity = 2.0

    world = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                density = noise.pnoise3(i / scale,
                                        j / scale,
                                        k / scale,
                                        octaves=octaves,
                                        persistence=persistence,
                                        lacunarity=lacunarity,
                                        repeatx=1024,
                                        repeaty=1024,
                                        repeatz=1024,
                                        base=42)
                world[i][j][k] = density

    #plot_dataframe(world)
    return world

In [4]:
terrain = generate_terrain()

In [5]:
terrain

array([[[0.        , 0.03258603, 0.0618329 , ..., 0.23677659,
         0.22908203, 0.23123303],
        [0.00326539, 0.03349958, 0.06179116, ..., 0.23497878,
         0.23004799, 0.23200431],
        [0.01080146, 0.03402521, 0.06090566, ..., 0.22904588,
         0.22667468, 0.22599192],
        ...,
        [0.06841178, 0.0551392 , 0.04430683, ..., 0.23193453,
         0.23161632, 0.2364942 ],
        [0.07256555, 0.06019788, 0.05213081, ..., 0.23015973,
         0.22912113, 0.23407307],
        [0.07774235, 0.06595241, 0.06102517, ..., 0.23576322,
         0.23336886, 0.23780766]],

       [[0.03258603, 0.06004842, 0.08003891, ..., 0.23510319,
         0.22625324, 0.22831614],
        [0.03446553, 0.06065301, 0.08070825, ..., 0.23241767,
         0.22653168, 0.22838107],
        [0.03747283, 0.06039054, 0.08373598, ..., 0.22481708,
         0.22253768, 0.22176737],
        ...,
        [0.07415111, 0.06217201, 0.05086714, ..., 0.24214798,
         0.24082111, 0.24386434],
        [0.0

# Funcion para pasar de matriz a dataframe

In [6]:
import pandas as pd
def using_multiindex(A, columns):
    shape = A.shape
    index = pd.MultiIndex.from_product([range(s)for s in shape], names=columns)
    df = pd.DataFrame({'density': A.flatten()}, index=index).reset_index()
    return df

In [7]:
dataframe_terrain = using_multiindex(terrain, list('xyz'))

In [8]:
dataframe_terrain

,x,y,z,density
0,0,0,0,0.000000
1,0,0,1,0.032586
2,0,0,2,0.061833
3,0,0,3,0.082446
4,0,0,4,0.097867
...,...,...,...,...
7995,19,19,15,0.377797
7996,19,19,16,0.366430
7997,19,19,17,0.352210
7998,19,19,18,0.334525


In [9]:
dataframe_terrain[['z']]

,z
0,0
1,1
2,2
3,3
4,4
...,...
7995,15
7996,16
7997,17
7998,18


# Dataframe a SDF

In [10]:
from math import sqrt


def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)


def getDistance(x, y, z, realInternalValue, internalValue = [0,1], columns = ['x','y','z','lineid']):
    coords = dataframe[columns]
    #print("internalValue: {} - realInternalValue: {}".format(internalValue, realInternalValue))
    
    realInternalValueIsAnInternalValue = internalValue[0] <= realInternalValue <= internalValue[1]
    #print(realInternalValueIsAnInternalValue)
    if realInternalValueIsAnInternalValue:
        # filtrar coords: quedarme con los distintos a internalValue
        coords = coords[~coords[columns[3]].between(internalValue[0], internalValue[1])]
    else:
        # filtrar coords: quedarme con los de igual a internalValue
        coords = coords[coords[columns[3]].between(internalValue[0], internalValue[1])]
    
    distances = coords.apply(lambda row: distanceSquared3D((x,y,z),(float(row[columns[0]]), float(row[columns[1]]), float(row[columns[2]]))), axis=1)
       
    min_distance = distances.min()
    if realInternalValueIsAnInternalValue:
        return -(min_distance)
    
    #print(min_distance)
    return min_distance
        
    
def dataframe_to_sdf(dataframe, internalValue = [0,1], columns = ['x','y','z','lineid']): #columns = [x, y, z, faultid, lineid]
    dataframe["distance"] = dataframe.apply(lambda x: getDistance(x[columns[0]], x[columns[1]], x[columns[2]], x[columns[3]], internalValue, columns), axis=1)
    return dataframe

In [11]:
#obtengo la densidad de uno de los puntos
dataframe_terrain.iloc[1].density

0.032586026936769485

In [12]:
import time

initial_time=time.time()
internalValue = [0.3,0.34] #dataframe_terrain.iloc[1].density
columns = ['x','y', 'z', 'density']
dataframe = dataframe_terrain
sdf = dataframe_to_sdf(dataframe_terrain, internalValue, columns)
end_time=time.time()
print("Calcularlo con 'map' tardo {} segundos".format(end_time-initial_time))
sdf

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
True
True
True
True
True
False
False
False
False
F

,x,y,z,density,distance
0,0,0,0,0.000000,15.811388
1,0,0,1,0.032586,15.524175
2,0,0,2,0.061833,15.297059
3,0,0,3,0.082446,15.132746
4,0,0,4,0.097867,15.033296
...,...,...,...,...,...
7995,19,19,15,0.377797,2.236068
7996,19,19,16,0.366430,1.732051
7997,19,19,17,0.352210,1.000000
7998,19,19,18,0.334525,-1.000000


# Funcion para crear esferas

In [13]:
len(dataframe)

8000

In [14]:
dataframe

,x,y,z,density,distance
0,0,0,0,0.000000,15.811388
1,0,0,1,0.032586,15.524175
2,0,0,2,0.061833,15.297059
3,0,0,3,0.082446,15.132746
4,0,0,4,0.097867,15.033296
...,...,...,...,...,...
7995,19,19,15,0.377797,2.236068
7996,19,19,16,0.366430,1.732051
7997,19,19,17,0.352210,1.000000
7998,19,19,18,0.334525,-1.000000


In [15]:
dataframe.iloc[0]

x            0.000000
y            0.000000
z            0.000000
density      0.000000
distance    15.811388
Name: 0, dtype: float64

In [16]:
from math import sqrt

def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)

def isInSphere(spheres, point):
    for sphere in spheres:
        if distanceSquared3D(point, sphere[0])< sphere[1]:
            return True
    return False

def spheres(dataframe, max_spheres): #dataframe = (x,y,z,density,distance)
    coordinates_spheres = [] #[(Point,r)]
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance > 0 and len(coordinates_spheres) < max_spheres):
            #print()
            coord = (dataframe.iloc[i].x, dataframe.iloc[i].y, dataframe.iloc[i].z)
            if(not isInSphere(coordinates_spheres, coord)):
                coordinates_spheres.append((coord, distance))

    return coordinates_spheres

def generateSpheres(dataframe, max_spheres, do_loop, use_brightness_level):
    spheresPoints = spheres(dataframe, max_spheres)
    # plot(circles, use_brightness_level, do_loop)
    return spheresPoints

In [17]:
spheres = generateSpheres(dataframe, 10, do_loop=False, use_brightness_level=True)

In [18]:
spheres

[((0.0, 0.0, 0.0), 15.811388300841896),
 ((0.0, 0.0, 16.0), 15.297058540778355),
 ((0.0, 14.0, 8.0), 12.165525060596439),
 ((2.0, 19.0, 19.0), 11.0),
 ((8.0, 19.0, 0.0), 8.54400374531753),
 ((10.0, 12.0, 15.0), 3.1622776601683795),
 ((11.0, 8.0, 9.0), 2.23606797749979),
 ((11.0, 10.0, 12.0), 2.8284271247461903),
 ((11.0, 11.0, 3.0), 2.23606797749979),
 ((11.0, 11.0, 18.0), 4.123105625617661)]

# Funcion para generar shader

In [19]:
def generateShader(circles):
    f = "float sphereSD(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }\nfloat sceneSDF(vec3 inputPoint) {\nreturn "
    circle = circles[0]
    prev = "sphereSD(inputPoint, vec3({0}, {1}, {3}), {2})".format(circle[0][0], circle[0][1], float(circle[1]), circle[0][2])
    for i in range(1,len(circles)):
        circle = circles[i]
        opt = "min(sphereSD(inputPoint, vec3({0}, {1}, {4}), {2}), {3})".format(circle[0][0], circle[0][1], float(circle[1]), prev, circle[0][2])
        prev = opt
    f += opt
    f += ";\n}"
    return f

In [20]:
generateShader(spheres)

'float sphereSD(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }\nfloat sceneSDF(vec3 inputPoint) {\nreturn min(sphereSD(inputPoint, vec3(11.0, 11.0, 18.0), 4.123105625617661), min(sphereSD(inputPoint, vec3(11.0, 11.0, 3.0), 2.23606797749979), min(sphereSD(inputPoint, vec3(11.0, 10.0, 12.0), 2.8284271247461903), min(sphereSD(inputPoint, vec3(11.0, 8.0, 9.0), 2.23606797749979), min(sphereSD(inputPoint, vec3(10.0, 12.0, 15.0), 3.1622776601683795), min(sphereSD(inputPoint, vec3(8.0, 19.0, 0.0), 8.54400374531753), min(sphereSD(inputPoint, vec3(2.0, 19.0, 19.0), 11.0), min(sphereSD(inputPoint, vec3(0.0, 14.0, 8.0), 12.165525060596439), min(sphereSD(inputPoint, vec3(0.0, 0.0, 16.0), 15.297058540778355), sphereSD(inputPoint, vec3(0.0, 0.0, 0.0), 15.811388300841896))))))))));\n}'